# Script para generar la solución del Primer Benchmark de la Competencia

## Si no presentaste aún tu primera solución, tenes la oportunidad de hacerlo en pocos Clicks!

**Hola! **  
  
Este Script es un Ejemplo de Procesamiento de los Datos, Modelado y Generación de una Solución.

Agregamos una pequeña explicación de lo que se hace en cada paso para ayudar a los que están comenzando ahora


### Importamos las librerías que vamos a utilizar

In [1]:
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import re

### Lectura de las Bases

Observamos los datos que tenemos disponibles en https://www.kaggle.com/c/interbank20/data

Vamos a trabajar ahora con todas las bases disponibles

In [2]:
rcc_train = pd.read_csv("../../data/raw/rcc_train.csv")
se_train = pd.read_csv("../../data/raw/se_train.csv", index_col="key_value")
censo_train = pd.read_csv("../../data/raw/censo_train.csv", index_col="key_value")
y_train = pd.read_csv("../../data/raw/y_train.csv", index_col="key_value").target

rcc_test= pd.read_csv("../../data/raw/rcc_test.csv")
se_test= pd.read_csv("../../data/raw/se_test.csv", index_col="key_value")
censo_test= pd.read_csv("../../data/raw/censo_test.csv", index_col="key_value")

### Vamos a trabajar ahora con la base de **RCC**:
* Discretizamos los días de atraso para poder manipularla mejor
* Hacemos tablas cruzadas sobre key_value y cada variable de interés, utilizando distintas funciones de agregación sobre el saldo del producto

In [3]:
bins = [-1, 0, 10, 20, 30, 60, 90, 180, 360, 720, float("inf")]
rcc_train["condicion"] = pd.cut(rcc_train.condicion, bins)
rcc_test["condicion"] = pd.cut(rcc_test.condicion, bins)

In [4]:
def makeCt(df, c, aggfunc=sum):
    try:
        ct = pd.crosstab(df.key_value, df[c].fillna("N/A"), values=df.saldo, aggfunc=aggfunc)
    except:
        ct = pd.crosstab(df.key_value, df[c], values=df.saldo, aggfunc=aggfunc)
    ct.columns = [f"{c}_{aggfunc.__name__}_{v}" for v in ct.columns]
    return ct

In [5]:
train = []
test = []
aggfuncs = [len, sum]
for c in rcc_train.drop(["codmes", "key_value", "saldo"], axis=1):
    print("haciendo", c)
    train.extend([makeCt(rcc_train, c, aggfunc) for aggfunc in aggfuncs])
    test.extend([makeCt(rcc_test, c, aggfunc) for aggfunc in aggfuncs])

haciendo condicion
haciendo tipo_credito
haciendo cod_instit_financiera
haciendo PRODUCTO
haciendo RIESGO_DIRECTO
haciendo COD_CLASIFICACION_DEUDOR


In [6]:
train[0]

,"condicion_len_(-1.0, 0.0]","condicion_len_(0.0, 10.0]","condicion_len_(10.0, 20.0]","condicion_len_(20.0, 30.0]","condicion_len_(30.0, 60.0]","condicion_len_(60.0, 90.0]","condicion_len_(90.0, 180.0]","condicion_len_(180.0, 360.0]","condicion_len_(360.0, 720.0]","condicion_len_(720.0, inf]"
key_value,,,,,,,,,,
0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51.0,4.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
358482,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358483,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358484,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
train[1]

,"condicion_sum_(-1.0, 0.0]","condicion_sum_(0.0, 10.0]","condicion_sum_(10.0, 20.0]","condicion_sum_(20.0, 30.0]","condicion_sum_(30.0, 60.0]","condicion_sum_(60.0, 90.0]","condicion_sum_(90.0, 180.0]","condicion_sum_(180.0, 360.0]","condicion_sum_(360.0, 720.0]","condicion_sum_(720.0, inf]"
key_value,,,,,,,,,,
0,-2.051825,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.540241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.363093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.193806,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-3.185017,-0.236501,NaN,-0.057877,-0.071022,-0.069141,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
358482,-0.195957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358483,-0.207356,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358484,-0.259724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
train[2]

,tipo_credito_len_6,tipo_credito_len_8,tipo_credito_len_9,tipo_credito_len_10,tipo_credito_len_11,tipo_credito_len_12,tipo_credito_len_13,tipo_credito_len_99
key_value,,,,,,,,
0,NaN,NaN,NaN,NaN,42.0,11.0,NaN,NaN
1,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,13.0,45.0,NaN,NaN
...,...,...,...,...,...,...,...,...
358482,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0
358483,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
358484,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN


In [9]:
train = pd.concat(train, axis=1)
test = pd.concat(test, axis=1)

In [11]:
train.head(10)

,"condicion_len_(-1.0, 0.0]","condicion_len_(0.0, 10.0]","condicion_len_(10.0, 20.0]","condicion_len_(20.0, 30.0]","condicion_len_(30.0, 60.0]","condicion_len_(60.0, 90.0]","condicion_len_(90.0, 180.0]","condicion_len_(180.0, 360.0]","condicion_len_(360.0, 720.0]","condicion_len_(720.0, inf]",...,COD_CLASIFICACION_DEUDOR_len_2,COD_CLASIFICACION_DEUDOR_len_3,COD_CLASIFICACION_DEUDOR_len_4,COD_CLASIFICACION_DEUDOR_len_5,COD_CLASIFICACION_DEUDOR_sum_0,COD_CLASIFICACION_DEUDOR_sum_1,COD_CLASIFICACION_DEUDOR_sum_2,COD_CLASIFICACION_DEUDOR_sum_3,COD_CLASIFICACION_DEUDOR_sum_4,COD_CLASIFICACION_DEUDOR_sum_5
key_value,,,,,,,,,,,,,,,,,,,,,
0,53.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.051825,NaN,NaN,NaN,NaN,NaN
1,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.540241,NaN,NaN,NaN,NaN,NaN
2,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.363093,NaN,NaN,NaN,NaN,NaN
3,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.193806,NaN,NaN,NaN,NaN,NaN
4,51.0,4.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,...,4.0,4.0,NaN,NaN,-2.938786,-0.182073,-0.251975,-0.246724,NaN,NaN
5,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.426465,NaN,NaN,NaN,NaN,NaN
6,33.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.263630,NaN,NaN,NaN,NaN,NaN
7,144.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-9.540998,NaN,NaN,NaN,NaN,NaN
8,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.943911,NaN,NaN,NaN,NaN,NaN


In [12]:
test.head(10)

,"condicion_len_(-1.0, 0.0]","condicion_len_(0.0, 10.0]","condicion_len_(10.0, 20.0]","condicion_len_(20.0, 30.0]","condicion_len_(30.0, 60.0]","condicion_len_(60.0, 90.0]","condicion_len_(90.0, 180.0]","condicion_len_(180.0, 360.0]","condicion_len_(360.0, 720.0]","condicion_len_(720.0, inf]",...,COD_CLASIFICACION_DEUDOR_len_2,COD_CLASIFICACION_DEUDOR_len_3,COD_CLASIFICACION_DEUDOR_len_4,COD_CLASIFICACION_DEUDOR_len_5,COD_CLASIFICACION_DEUDOR_sum_0,COD_CLASIFICACION_DEUDOR_sum_1,COD_CLASIFICACION_DEUDOR_sum_2,COD_CLASIFICACION_DEUDOR_sum_3,COD_CLASIFICACION_DEUDOR_sum_4,COD_CLASIFICACION_DEUDOR_sum_5
key_value,,,,,,,,,,,,,,,,,,,,,
0,208.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-6.890093,NaN,NaN,NaN,NaN,NaN
1,56.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.713460,-0.295531,NaN,NaN,NaN,NaN
2,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-5.011280,NaN,NaN,NaN,NaN,NaN
3,175.0,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,11.0,7.0,NaN,NaN,-8.134954,-0.402380,-0.662106,-0.324942,NaN,NaN
4,26.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.073227,NaN,NaN,NaN,NaN,NaN
5,111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,8.880288,NaN,NaN,NaN,NaN,NaN
6,209.0,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.703643,-0.459128,NaN,NaN,NaN,NaN
7,48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.703159,NaN,NaN,NaN,NaN,NaN
8,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-3.874054,NaN,NaN,NaN,NaN,NaN


### Incorporamos la Información adicional existente en las tablas socio económicas y del censo. Es un simple join porque ambas tienen key_value únicos
#### Por el momento no incorporamos la información tributaria porque requiere un tratamiento más complejo que queda para futuras revisiones

In [13]:
train = train.join(censo_train).join(se_train)
test = test.join(censo_test).join(se_test)

### Por la naturaleza de las variables creadas, nos aseguramos que solo se utilicen variables existentes en ambos conjuntos de datos (train y test)

In [14]:
keep_cols = list(set(train.columns).intersection(set(test.columns)))
train = train[keep_cols]
test = test[keep_cols]
len(set(train.columns) - set(test.columns)) , len(set(test.columns) - set(train.columns))

(0, 0)

In [15]:
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))
train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_-]+', '', x))

### Entrenamiento del Modelo

Para entrenar nuestro modelo vamos a usar LightGBM

In [16]:
folds = [train.index[t] for t, v in KFold(5).split(train)]
test_probs = []
train_probs = []
fi = []
for i, idx in enumerate(folds):
    print("*"*10, i, "*"*10)
    Xt = train.loc[idx]
    yt = y_train.loc[Xt.index]

    Xv = train.drop(Xt.index)
    yv = y_train.loc[Xv.index]
    
    learner = LGBMClassifier(n_estimators=1000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)], verbose=50)
    test_probs.append(pd.Series(learner.predict_proba(test)[:, -1], index=test.index, name="fold_" + str(i)))
    train_probs.append(pd.Series(learner.predict_proba(Xv)[:, -1], index=Xv.index, name="probs"))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))
          
test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)
fi = pd.concat(fi, axis=1).mean(axis=1)

print("*" * 21)
print("roc auc estimado: ", roc_auc_score(y_train, train_probs.loc[y_train.index]))

********** 0 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.829126	training's binary_logloss: 0.31682	valid_1's auc: 0.824671	valid_1's binary_logloss: 0.322677
[100]	training's auc: 0.842425	training's binary_logloss: 0.307142	valid_1's auc: 0.830032	valid_1's binary_logloss: 0.317725
Early stopping, best iteration is:
[103]	training's auc: 0.843004	training's binary_logloss: 0.306766	valid_1's auc: 0.830053	valid_1's binary_logloss: 0.317671
********** 1 **********
Training until validation scores don't improve for 10 rounds
[50]	training's auc: 0.829145	training's binary_logloss: 0.317248	valid_1's auc: 0.824082	valid_1's binary_logloss: 0.320768
[100]	training's auc: 0.842633	training's binary_logloss: 0.307434	valid_1's auc: 0.82932	valid_1's binary_logloss: 0.31611
[150]	training's auc: 0.852004	training's binary_logloss: 0.301297	valid_1's auc: 0.830228	valid_1's binary_logloss: 0.315171
Early stopping, best iteration is:
[148]	tra

In [17]:
fi.sort_values(ascending=False).to_frame()

,0
RIESGO_DIRECTO_len_1,0.056761
tipo_credito_len_11,0.035533
edad,0.034513
RIESGO_DIRECTO_sum_1,0.033829
tipo_credito_sum_12,0.033212
...,...
gas_ali_d_18,0.000000
gas_sal_b_18,0.000000
gas_ali_b_18,0.000000
gas_trans_b_18,0.000000


### Guardado del modelo para hacer la presentación

Finalmente creamos el archivo CSV que podemos subir como nuestra Solución a la competencia

Empezá con este archivo y luego podes seguir mejorándolo a ver si subís en posiciones!

In [12]:
test_probs.name = "target"
test_probs.to_csv("benchmark1.csv")